**Bioinformatique : Rendu_5** <Br>
Hajar Hajji

In [27]:
# ---------------------         Classe Noeud         --------------------- #

class Noeud:
    """
    classe modélisant un noeud (interne ou externe) d'un arbre
    """

    def __init__(self, nom = None):
        
        #nom du noeud (None pour un noeud interne)
        self.nom = nom
        
        #séquence du noeud (None pour un noeud interne)
        self.sequence = None        
        
        #fils du noeud (liste vide pour une feuille)
        self.fils = list()
        
        #père du noeud (None pour la racine)
        self.pere = None        
    
    def est_une_feuille(self):
        """
        méthode renvoyant True si le noeud est une feuille, False sinon
        """
        
        if self.fils != []:
            return False
        return True
    
    def ajout_sequences(self, sequences):
        """
        méthode de mémorisation des séquences des feuilles de l'arbre dont 
        le noeud est la racine
        """
        #cas d'une feuille
        if self.est_une_feuille():
            self.sequence = sequences[self.nom]
        #cas d'un noeud interne
        else :
            for enfant in self.fils:
                enfant.ajout_sequences(sequences)
        return
    
    def calcul_nombre_pas(self, position):
        """
        méthode calculant le nombre de pas pour l'arbre pour le site d'indice
        "position" de l'alignement mutliple selon la méthode "d'interunion"
        """
        def intersection(interunion_fils_1, interunion_fils_2):
            """
            l'intersection de 2 ensembles de caractères
            """
            intersection=""
            
            for ch in interunion_fils_1:
                if ch in interunion_fils_2:
                    intersection+=ch
                    
            return intersection
        
        #cas d'une feuille
        if self.est_une_feuille():
            return (0,self.sequence[position])
        
        #cas d'un noeud interne
        else:
            #1er fils
            nombre_pas_fils_1, interunion_fils_1 = self.fils[0].calcul_nombre_pas(position)
            #2ème fils
            nombre_pas_fils_2, interunion_fils_2 = self.fils[1].calcul_nombre_pas(position)
            #calcul de l'intersection
            intersection = intersection(interunion_fils_1, interunion_fils_2)
            #si l'intersection est vide
            if len(intersection) == 0:
                nombre_pas = nombre_pas_fils_1 + nombre_pas_fils_2 + 1
                union = interunion_fils_1 + interunion_fils_2
                return nombre_pas, union
            else:
                nombre_pas = nombre_pas_fils_1 + nombre_pas_fils_2
                return nombre_pas, intersection
    
    def calcul_nombre_total_pas(self, longueur_sequence, sites_informatifs):
        """
        méthode calculant le nombre de pas pour l'arbre selon la méthode 
        "d'interunion"
        """
        nombre_total_pas = 0
        
        #calcul du nombre de pas...
        for i in range(longueur_sequence):
            # ... pour chaque site informatif
            if sites_informatifs[i]:
                nombre_pas, intersection = self.calcul_nombre_pas(i)
                nombre_total_pas += nombre_pas
        
        #renvoi du nombre total de pas
        return nombre_total_pas

In [28]:
# ---------------         lecture  de l’arbre phylogénétique        --------------- #

def lecture_arbre(fichier_arbre):
    """
    fonction permettant de lire un arbre au format newick et de le stocker 
    dans une structure de données de type arbre
    """  

    fichier_entree = open(fichier_arbre, 'r')    
    
    #lecture de l'unique ligne contenant l'arbre
    ligne = fichier_entree.readline()
    
    #vérification du format du fichier
    if ligne[0] != '(':
        print('Votre fichier n\'est pas au format newick')
        return None
    if ligne[-1] != ';' and ligne[-2] != ';':
        print('Votre fichier n\'est pas au format newick')
        return None        
    
    #arbre
    if ligne[-1] == '\n':
        arbre = ligne[:-2]
    else:
        arbre = ligne[:-1]
    
    #construction de l'arbre
    racine = None
    noeud_courant = None
    i = 0
    while i < len(arbre):
        if arbre[i] == '(':
            if racine == None:
                racine = Noeud()
                noeud_courant = racine
            else:
                noeud = Noeud()
                noeud_courant.fils.append(noeud)
                noeud.pere = noeud_courant
                noeud_courant = noeud
            i += 1
        elif arbre[i] == ',' or arbre[i] == ')':
            noeud_courant = noeud_courant.pere
            i += 1
        else:
            nom = ''
            while arbre[i] != ',' and arbre[i] != ')':
                nom += arbre[i]
                i += 1
            noeud = Noeud(nom)
            noeud_courant.fils.append(noeud)
            noeud.pere = noeud_courant
            noeud_courant = noeud
    
    #renvoie la racine de l'arbre
    return racine

In [29]:
# ---------------         lecture de l’alignement multiple        --------------- #

def lecture_sequences(fichier_sequences):
    """
    fonction permettant de lire un alignement de séquences au format fasta et 
    de le stocker dans un dictionnaire nom/séquence
    """
    
    fichier_entree=open(fichier_sequences, 'r')    
    
    #lecture de toutes les lignes du fichier
    lignes=fichier_entree.readlines()
    
    #vérification du format du fichier
    if lignes[0][0]!= '>':
        print('Votre fichier n\'est pas au format fasta')
        return None
    
    #récupération des séquences
    sequences=dict()
    i=0
    while i < len(lignes):
        nom=lignes[i][1:-1]
        i+= 1
        seq = ''
        while i < len(lignes) and lignes[i][0]!= '>':
            seq+= lignes[i][:-1]
            i+= 1
        sequences[nom]=seq
    
    fichier_entree.close()
    
    #longueur des séquences
    longueur_sequence=len(list(sequences.values())[0])
    
    #renvoi du dictionnaire et de la longueur des séquences
    return sequences,longueur_sequence

In [30]:
# def recherche_sites_informatifs(sequences, longueur_sequence):
#     """
#     fonction renvoyant une liste de booléens indiquant si chaque site de 
#     l'alignement est ou non informatif
#     """
    

In [31]:
def recherche_sites_informatifs(sequences, longueur_sequence):
    '''
    fonction renvoyant une liste de booléens indiquant si chaque site de 
    l'alignement est ou non informatif
    '''
    
    # liste de l'état informatif des sites considérés par défaut comme non 
    # informatifs
    sites_informatifs = [False]*longueur_sequence
    
    # parcours de tous les sites de l'alignement
    for i in range(longueur_sequence):
        # dictionnaire contenant le nombre d'occurence de chaque résidu 
        # présent au moins une fois au site courant
        dico = dict()
        # remplissage du dictionnaire
        for nom in sequences:
            aa = sequences[nom][i]
            if aa not in dico:
                dico[aa] = 1
            else:
                dico[aa] += 1
        # détermination du caractère informatif du site courant
        if len(dico) > 1:
            # recherche du nombre de résidus présents au moins 2 fois
            cpt = 0
            for aa in dico:
                if dico[aa] > 1:
                    cpt += 1
            # cas où le site est informatif
            if cpt > 1:
                sites_informatifs[i] = True
    
    # renvoi de la liste
    return sites_informatifs

In [32]:
# ---------------        Programme principal          --------------- #

###arbre phylogénétique
print("Topologie 1 :")
fichier_arbre="topologie_1.txt"
racine=lecture_arbre(fichier_arbre)

###alignement multiple
fichier_sequences="alignement_multiple_cytochrome-b.fasta"
sequences,longueur_sequence=lecture_sequences(fichier_sequences)

###mémorisation des séquences dans l'arbre
racine.ajout_sequences(sequences)

# --- analyse par la méthode du maximum de parcimonie --- #

###recherche des sites non informatifs
sites_informatifs=recherche_sites_informatifs(sequences, longueur_sequence)

###calcul du nombre de pas
nombre_total_pas=racine.calcul_nombre_total_pas(longueur_sequence, 
                                                  sites_informatifs)
print("Nombre de pas : %s" % (nombre_total_pas))

print()
print("---------------------------------------------------------")
print()

###arbre phylogénétique 2
print("Topologie 2 :")
fichier_arbre_2="topologie_2.txt"
racine_2=lecture_arbre(fichier_arbre_2)

###mémorisation des séquences dans l'arbre
racine_2.ajout_sequences(sequences)

###recherche des sites non informatifs
sites_informatifs_2=recherche_sites_informatifs(sequences, longueur_sequence)

###calcul du nombre de pas
nombre_total_pas_2=racine_2.calcul_nombre_total_pas(longueur_sequence, 
                                                  sites_informatifs_2)
print("Nombre de pas : %s" % (nombre_total_pas_2))

print()
print("---------------------------------------------------------")
print()

###arbre phylogénétique 3
print("Topologie 3 :")
fichier_arbre_3="topologie_3.txt"
racine_3=lecture_arbre(fichier_arbre_3)

###alignement multiple
sequences_3,longueur_sequence_3=lecture_sequences(fichier_sequences)

###mémorisation des séquences dans l'arbre
racine_3.ajout_sequences(sequences)

###recherche des sites non informatifs
sites_informatifs_3=recherche_sites_informatifs(sequences,longueur_sequence)

###calcul du nombre de pas
nombre_total_pas_3=racine_3.calcul_nombre_total_pas(longueur_sequence, 
                                                  sites_informatifs_3)
print("Nombre de pas : %s" % (nombre_total_pas_3))

Topologie 1 :
Nombre de pas : 97

---------------------------------------------------------

Topologie 2 :
Nombre de pas : 99

---------------------------------------------------------

Topologie 3 :
Nombre de pas : 101
